In [1]:
# Install necessary packages
%pip install prophet
%pip install scikit-learn
%pip install behave
%pip install pandas
%pip install numpy
%pip install statsmodels
%pip install seaborn


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
import itertools
# from google.colab import files
import io
from sklearn.model_selection import TimeSeriesSplit
import unittest
import seaborn as sns
import time

import kellys

data = kellys.load_data()
train_data, test_data, train_exog, test_exog = kellys.preprocess(data)
sarima_results = kellys.train(train_data, train_exog)
print(sarima_results)

Loading data
Exogenous variables data types after conversion:
 Store ID                           int64
Product ID                         int64
Country_Mexico                     int64
Country_USA                        int64
Product Category_Clothing          int64
Product Category_Electronics       int64
Product Category_Home & Kitchen    int64
Product Category_Sports            int64
dtype: object
Training data: 7134 records
Testing data: 1784 records
Training exogenous data types:
Store ID                           int64
Product ID                         int64
Country_Mexico                     int64
Country_USA                        int64
Product Category_Clothing          int64
Product Category_Electronics       int64
Product Category_Home & Kitchen    int64
Product Category_Sports            int64
dtype: object
All training exogenous data columns are numeric.


/Users/prakash/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/prakash/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/prakash/Library/Python/3.9/lib/python/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                     SARIMAX Results                                      
Dep. Variable:                         Units Sold   No. Observations:                 7134
Model:             SARIMAX(1, 1, 1)x(1, 1, 1, 12)   Log Likelihood              -21679.502
Date:                            Sun, 16 Jun 2024   AIC                          43385.004
Time:                                    20:10:13   BIC                          43474.299
Sample:                                01-01-2000   HQIC                         43415.750
                                     - 07-13-2019                                         
Covariance Type:                              opg                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Store ID                           -0.0506      0.042     -1.216      0.

In [13]:
print(sarima_results)

None


In [11]:
import pandas as pd

def preprocess_future_exog(future_exog):
    # Convert categorical variables into dummy/indicator variables
    future_exog = pd.get_dummies(future_exog, drop_first=True)

    # Convert boolean columns to integers
    bool_columns = future_exog.select_dtypes(include=['bool']).columns
    future_exog[bool_columns] = future_exog[bool_columns].astype(int)

    # Ensure the future_exog has the same columns as train_exog
    future_exog = future_exog.reindex(columns=train_exog.columns, fill_value=0)

    # Ensure all exog variables are numeric
    future_exog = future_exog.apply(pd.to_numeric, errors='coerce')

    # Check the data types of the future exogenous variables
    # print("Future exogenous variables data types after conversion:\n", future_exog.dtypes)

    return future_exog

def forecast(future_dates, future_exog):
    future_exog = preprocess_future_exog(future_exog)
    # Forecast future values
    forecast = sarima_results.get_forecast(steps=len(future_dates), exog=future_exog)
    forecast_mean = forecast.predicted_mean

    # Set the index of the forecast to the future dates
    forecast_mean.index = future_dates

    # Print the forecasted sales
    print(forecast_mean)


In [12]:
# Define future dates for which we want to forecast
future_dates = pd.date_range(start='2025-01-10', end='2025-01-20', freq='D')

# Create a DataFrame for the future dates with the same structure as the exogenous data
future_exog = pd.DataFrame({
    'Country': ['USA'] * len(future_dates),  # Replace with desired countries
    'Store ID': [4] * len(future_dates),    # Replace with desired store IDs
    'Product ID': [15] * len(future_dates), # Replace with desired product IDs
    'Product Category': ['Home & Kitchen'] * len(future_dates)  # Replace with desired product categories
}, index=future_dates)

forecast(future_dates, future_exog)

AttributeError: 'NoneType' object has no attribute 'get_forecast'

In [7]:
# Define future dates for which we want to forecast
future_dates = pd.date_range(start='2025-01-10', end='2025-01-20', freq='D')

# Create a DataFrame for the future dates with the same structure as the exogenous data
future_exog = pd.DataFrame({
    'Country': ['Mexico'] * len(future_dates),  # Replace with desired countries
    'Store ID': [5] * len(future_dates),    # Replace with desired store IDs
    'Product ID': [16] * len(future_dates), # Replace with desired product IDs
    'Product Category': ['Home & Kitchen'] * len(future_dates)  # Replace with desired product categories
}, index=future_dates)

forecast(future_dates, future_exog)

AttributeError: 'NoneType' object has no attribute 'get_forecast'